To do to make this a real tutorial:

- use Twarc programmatically to download json
- nicer file path setup/config
- fill in initial instructions

Run this page locally as a Jupyter notebook by downloading it... (*instructions*)

(*Python3 and jupyter instructions*)

## Setup and configuration

Libraries used:

- `tidy_tweet`: link to installation instructions
- [`pathlib`](https://docs.python.org/3/library/pathlib.html): file path utility, does not require installation as is included in the Python Standard Library
- [`sqlite3`](https://docs.python.org/3/library/sqlite3.html): Python interface for [SQLite](https://sqlite.org) databases, does not require installation as is included in the Python Standard Library

*tabulate and Ipython explanations - maybe import them in the section they're used*

In [ ]:
from tidy_tweet import initialise_sqlite, load_twarc_json_to_sqlite
import sqlite3
from pathlib import Path
from IPython.display import Markdown
from tabulate import tabulate

Configure the paths to where the data is stored:

In [ ]:
tweet_json_file = Path('ObservatoryTeam.jsonl')
tweet_database = Path('ObservatoryTeam_tutorial.db')

## Use `tidy_tweet` to parse the tweets and load them into a database

Initialise the database to store the tweet data in:

In [ ]:
initialise_sqlite(tweet_database)

Now at the path you specified in `tweet_database` will be an SQLite database file with empty tables in it, ready to start loading some tweets into!

Let's load the tweet file:

In [ ]:
load_twarc_json_to_sqlite(tweet_json_file, tweet_database)

The number returned by `load_twarc_json_to_sqlite()` tells you how many *pages* of Twitter API response results were parsed. The tweets contained in these results pages are now loaded into your database for easy querying!

## Explore the tweets in the database using SQL

Query time!

How many tweets do we have in the database?

In [ ]:
with sqlite3.connect(tweet_database) as connection:
    db = connection.cursor()

    db.execute("select count(*) from tweet")

    print(f"There are {db.fetchone()[0]} tweets in the database!")

In [ ]:
with sqlite3.connect(tweet_database) as connection:
    db = connection.cursor()

    # Find the 10 dates with the most tweets created within this collection
    db.execute("""
        select 
            date(created_at) as date_tweeted, 
            count(*) as num_tweets 
        from tweet 
        group by date_tweeted 
        order by num_tweets desc 
        limit 10
        """)
    
    top_10_days = db.fetchall()


We could be querying the database directly with a client (*explain*), but since we're doing this in a Jupyter notebook presumably we want to show our results nicely or do programmatic things with them.

Let's have a look at a table of our results:

In [ ]:
Markdown(tabulate(top_10_days, headers=["Date", "Number of tweets"], tablefmt="github"))

Other examples:

- top hashtags, then pick one and read the tweets
- domains linked to above a threshold number of times and number of distinct pages linked to for each of them
- add annotation column and mark tweets/hashtags of interest? Or add category column to user table and add categories then group/filter tweets by that?
- something involving programmatic manipulation in python?